Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def compound_interest_widget(xMin_init = 0, xMax_init = 1.1, yMin_init = 0, yMax_init = 3, 
                  numTermsStr_init = '1', expFlag_init = False):
    
    # Declare widgets for interactive input
    xMin_slider = widgets.FloatSlider(min=0,
                                 max=1.1,
                                 description=r'Min $x$:',
                                 value = xMin_init,
                                 continuous_update =False)
    xMax_slider = widgets.FloatSlider(min=0,
                                 max=1.1,
                                 description=r'Max $x$:',
                                 value = xMax_init,
                                 continuous_update =False)
    yMin_slider = widgets.FloatSlider(min=0,
                                 max=3,
                                 description=r'Min $y$:',
                                 value = yMin_init,
                                 continuous_update =False)
    yMax_slider = widgets.FloatSlider(min=0,
                                 max=3,
                                 description=r'Max $y$:',
                                 value = yMax_init,
                                 continuous_update =False)
    numTermsStr_text = widgets.Text(value = numTermsStr_init,
                                placeholder='Enter numbers separated by commas',
                                description='N° terms:',
                                disabled=False)
    expFlag_check = widgets.Checkbox(value = expFlag_init,
                                   description='Include Exponential',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((xMin_slider,'value'),(xMax_slider,'min'))
    widgets.jslink((xMax_slider,'value'),(xMin_slider,'max'))
    widgets.jslink((yMin_slider,'value'),(yMax_slider,'min'))
    widgets.jslink((yMax_slider,'value'),(yMin_slider,'max'))

    def compound_interest_plot(xMin, xMax, yMin, yMax, numTermsStr, expFlag):

        numTermsList = numTermsStr.split(',')

        if len(numTermsList) > 6:
            numTermsList = numTermsList[0:6]

        colors = ['b','r','g','m','c','y']

        # Create figure, plot compounded functions
        fig, ax = plt.subplots(figsize=(20,10))
        mrkrSize = 2*rcParams['lines.markersize'] ** 2

        for index, item in enumerate(numTermsList):
            
            # Process entry if item is valid
            if not item == '' :
                numTerms = int(item)

                step = 1/numTerms
                rate = 1 + step
                K,T = 1,0
                ax.scatter(T, K, s=mrkrSize, c='b', alpha=0.6)
                col = colors[index]

                for n in range(numTerms):
                    if n == 0:
                        termLabel = r'N$^\circ$ terms: {:d}'.format(numTerms)
                    else:
                        termLabel = None
                    ax.scatter(T+step, K*rate, s=mrkrSize, c=col, alpha=0.6)
                    ax.plot([T,T+step], [K,K], col, linewidth=2, alpha=0.6, label=termLabel)
                    ax.plot([T+step,T+step], [0,K*rate], col+'--', linewidth=2, alpha=0.6)
                    T+=step
                    K*=rate

                ax.plot([0,T],[K,K],col+'--',linewidth=1)
                ax.annotate(r'Final value = ${:.4f}$'.format(K),[0,K], 
                            xytext = [0.05,K+0.1], xycoords ='data', fontsize = 25, 
                            clip_on = True)

        # Plot the exponential if requested
        if expFlag is True:
            x = np.arange(0,1,1/500)
            y = np.exp(x)
            ax.plot(x, y,'k', linewidth=2, label=r'$\quad y = a^x$')

        # Add legend and format axes to look nice
        ax.legend(loc='lower center', frameon=False,prop={'size':20},ncol=6,
                   bbox_to_anchor=(0.5, -0.25))

        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = yMax, bottom = yMin)
        ax.set_xlim(right = xMax, left = xMin)
        ax.spines['bottom'].set_position('zero')
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_position('zero')
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$T$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$Assets$', fontdict = {'fontsize': 25},
                      position=(0, 1.05), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(compound_interest_plot, {'xMin': xMin_slider,
                                                            'xMax': xMax_slider,
                                                            'yMin': yMin_slider,
                                                            'yMax': yMax_slider,
                                                            'numTermsStr': numTermsStr_text,
                                                            'expFlag' : expFlag_check})

    output = widgets.VBox([out,
                  widgets.HBox([xMin_slider,
                               xMax_slider,
                               yMin_slider,
                               yMax_slider]),
                  widgets.HBox([numTermsStr_text, 
                                expFlag_check])])
    display(output)    
    

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Deep(er) dive: Where does $e$ come from?

As we discussed in the videos, the natural logarithm $y = \ln x$ and its inverse, the exponential function $y=e^x$ both rely on this 'special' number called $e$. Like $\pi$, $e$ is a mathematical constant that is irrational, i.e. it cannot be expressed as a ratio, and it's value is approximately 2.71828. 

The question we raised in the video was essentially as follows
1. All logarithms have this nice property that $\log(xy) = \log(x) + \log(y)$. You can pick any base you want and this property remains
2. Some bases make very practical sense, like base 10 (our number system is base 10) or base 2 (for computer/software engineering)

Given points 1 and 2, why on earth would you pick this strange number as your preferred base? The notebook attempts to answer this while giving you an insight into the history of the concept.

#### The problem of compounding interest in finance

The number $e$ is named after genius Swiss mathematician Leonhard Euler (1707-1783), who worked a lot with its theoretical properties (Euler in particular proved it was an irrational number). He is the one who popularised the notation $e$ for this constant, which means the guy obviously thought he was a big deal! Euler did not discover $e$, however, that credit should go to Jacob Bernoulli, another giant swiss mathematician, who found out an expression for $e$ in 1683 (long before Euler). The route he used to find the value of $e$ was exactly the one we will use here, and had to do with finding the most profitable strategy for earning interest on a loan.

Banking and finance massively expanded in post-renaissance Europe, as larger, more expensive and more risky enterprises were set off, such as the rise of manufacturing, financing shipping expeditions to open new trade routes, etc. But the maths behind finance (especially probability theory) wasn't yet understood, and lending still relied on rules of thumb.

Bernoulli essentially started with this setting: suppose you invest in a business or a security that doubles your initial investment after a time $T$. Most examples of this thinking will use 1 year, but actually (a) that's not very realistic and (b) we don't really care how long $T$ is, only that it exists. Even if the annual interest rate is very low, eventually you will double your money!

We can plot this below, and it produces a boring diagram. You give away your money, and after $T$ time has passed, you get your money back with 100% interest, so your wealth doubles at that point.

In [3]:
compound_interest_widget()

Bernoulli then asked this crucial question: ***would it be a more profitable strategy to lend at the same rate for half the time***? In other words, you lend at the same rate, but ask for the loan to be repaid, with interest, at time $T/2$. You then re-lend all of what you received (capital plus interest) on the same terms, again for $T/2$. ***Are you better off?***

The answer is yes, and it is easy to see why (diagram below). For every pound you lend initially, you receive £1.5 at the halfway point $T/2$. £1 of this is the repayment of the initial capital, 50p is the interest. Of course, because you lent the money for half the time, you only get half the interest. But crucially, when you re-lend the money again immediately at the same interest rate, you are now lending £1.50, not £1. So for the second half of the diagram, the same interest rate applies to a larger amount of capital! When you get paid back the second time, you receive £2.25: £1.50 of that is the initial capital you lent, 75p of interest.

In [4]:
compound_interest_widget(numTermsStr_init = '1,2')

We can write this down as an equation. Let $A_1$ be the amount you get for each pound invested in a single loan $T$ (our first example). This is easy, you get £1 of capital and £1 of interest:

$$A_1 = 1 + 1 = 2$$

Let's do the same for when we lend twice over the period $T$, and call it $A_2$:

$$A_2 \quad = \quad \left(1+\frac{1}{2}\right)\left(1+\frac{1}{2}\right) \quad = \quad \left(1+\frac{1}{2}\right)^2 \quad = \quad 2.25$$

The first $1+1/2$ bracket tells us how much we have to re-invest when the first loan gets paid back (£1.50), the second bracket corresponds to the growth in value from the second loan.

Bernoulli probably wasn't the first to figure out that you would benefit from getting some the interest paid early, but was the first to 'do the math'. The obvious question at this point is: ***can we improve on this result by making the loans even shorter and lending more often?***

In [5]:
compound_interest_widget(numTermsStr_init = '1,2,4')

The diagram above shows what happens when you break $T$ down into four periods. At $T/4$, when the first round of lending is repaid, you only get 25p per pound of capital (you are only lending for 1/4 of the original period), but like before, getting paid some of the interest early so that you can increase the capital you are lending pays off in the long run.

Using the same logic as before, the amount you get for breaking up $T$ into 4 successive rounds of lending can be written as:

$$A_4 \quad = \quad \left(1+\frac{1}{4}\right)^4 \quad \approx \quad 2.4414$$

In general, breaking the loan period down into $n$ rounds of lending, each returning $1/n$ of effective interest is given by the following formula:

$$A_n \quad = \quad \left(1+\frac{1}{n}\right)^n$$

You may have noticed, however, that while lending four times has increased the gain (so we are better off), it is less effective than the first time round. When we first split the lending period in two, we were 25p better off. Splitting into two again (so 4 loans) gained us 19.14p. So while this works, there is no 'magic money tree'. 

Bernoulli noticed this too, and so the ***real*** question that he answered was: what happens to the value of $A_n$ as you keep increasing $n$? Could you keep increasing your gains by lending for increasingly short periods of time, each earning smaller and smaller amounts?

In [6]:
compound_interest_widget(numTermsStr_init = '50',expFlag_init = True)

As you can see for the example above, where you slice the initial time period $T$ into 50 successive loans, the step curve you get really looks like it is converging to the exponential function $e^x$. And the value of your capital after a unit amount of time $T$ is starting to look suspiciously close to the number I gave above. This is no surprise. Bernoulli showed that the value of $A_n$ when $n$ tends to infinity is actually $e$. More formally:

$$\mathop\lim_{n \to \infty}A_n \quad = \quad \mathop\lim_{n \to \infty}\left(1+\frac{1}{n}\right)^n \quad = \quad e$$

So to visualise this in our example, if you have a loan that pays £2 after $T$ time has elapsed for every pound you invest, the maximum gain you can get by compounding the interest continuously instead of paying it in one lump-sum at the end is £2.72.  

#### Wrapping up - A natural framework for growth

This illustration shows that the exponential function is the natural way of tracking growth when two conditions are met:
1. The rate of growth is fixed over time. In our example this was the fixed rate of interest.
2. Growth happens continuously over time, and not in discrete jumps that happen at specific points in time. In our example this was the slicing of the loans into shorter and shorter periods of time, and in the limit, into a continuous process.

As you an imagine, this has applications in a lot of places: even if a lot of biological or economic growth processes are actually discrete (like a cell dividing into two, or a firm being created), the individual units are so small, and there are so many of them that we can treat the overall process as continuous. The general form of the exponential in those cases is:

$$A(t) = A_0 e^{rt}$$

Where $A_0$ is the initial value of the quantity that is growing (or decaying if $r<0$), $r$ is the fixed rate of growth or decay (where 10% would be equal to 0.1) and $t$ is the amount of time elapsed. Our example of Mr Bernoulli's derivation simply had a combination of $r$ and $t$ that would be equal to one.